# Mackey Glass Benchmark Tutorial

This tutorial aims to provide an insight on how the NeuroBench framework is organized and how you can use it to benchmark your own models!

## About Mackey Glass:

### Dataset:
The Mackey Glass task is a chaotic function prediction task. Contrary to the other tasks in NeuroBench, the Mackey Glass dataset is synthetic. Real-world data can be high-dimensional and require large networks to achieve high accuracy, presenting challenges for solution types with limited I/O support and network capacity, such as mixed-signal prototype solutions. The Mackey Glass dataset is a one-dimensional non-linear time delay differential equation, where the evoluiton of the signal can be altered by a number of different parameters. These parameters are defined in NeuroBench. 
<!-- $$ dx \over dt = \beta {x(t-\tau)} \over {1 + x(t-\tau)^n} - \gamma x(t)
$$ -->
$$ \frac{dx}{dt} = \frac{\beta x(t-\tau)}{1 + x(t-\tau)^n} - \gamma x(t) $$

### Benchmark Task:
The task is a sequence-to-sequence prediction problem, similar to the primate reaching task, included in NeuroBench. The input sequence x is used to predict the future values of the same sequence, y(t) = x(t). The input data is passed at a timestep of $\Delta$ t, and the performance of the system is be tested in a multi-horizon prediction setting, where future values of the sequence are predicted at a rate of $\Delta$ t. The task’s difficulty is be varied by adjusting the ratio between the integration time step $\Delta$ t and the timescale $\tau$ of the underlying dynamics.




First we will import the relevant libraries. These include the datasets, preprocessors and accumulators. To ensure your model to be compatible with the NeuroBench framework, we will import the wrapper for snnTorch models. This wrapper will not change your model. Finally, we import the Benchmark class, which will run the benchmark and calculate your metrics.

In [ ]:
import torch

from torch.utils.data import Subset, DataLoader

import pandas as pd

from neurobench.datasets import MackeyGlass
from neurobench.models import TorchModel
from neurobench.benchmarks import Benchmark


For this tutorial, we will make use of the example architecture that is included in the NeuroBench framework.

In [2]:
# this is the network we will be using in this tutorial
from neurobench.examples.mackey_glass.echo_state_network import EchoStateNetwork

The Mackey Glass task is a synthetic dataset that is generated upon calling the MackeyGlass function. The parameters of the Mackey Glass function have to be passed by the user. These parameters define the output sequence that is generated. The NeuroBench framework provides the parameters that can be used for obtaining the data.

In [ ]:
# Mackey Glass parameters
mg_parameters_file="neurobench/datasets/mackey_glass_parameters.csv"
mg_parameters = pd.read_csv(mg_parameters_file)


FileNotFoundError: [Errno 2] No such file or directory: '/home/korneel/NeuroBench/algorithms_benchmarks/neurobench/examples/gsc/data/speech_commands/tmpy8couawl'

Next, lwe load the hyperparameters of the echo state networks that are found via random search.

In [ ]:
esn_parameters = pd.read_csv("echo_state_network_hyperparameters.csv")


No preprocessors are used in this task.

In [ ]:
preprocessors = []
postprocessors = []

The Mackey Glass task contains 14 series with varying complexity. Every series is ran a number of times.

In [ ]:
# benchmark run over 14 different series
sMAPE_scores = []

# Number of simulations to run for each time series
repeat = 10

We shift the time series by 0.5 of its Lyapunov times for each independent run:

In [ ]:
# Shift time series by 0.5 of its Lyapunov times for each independent run 
start_offset_range = torch.arange(0., 0.5*repeat, 0.5) 

With everything set up, we are ready to start the benchmark. Remember that we repeat the simulation of 14 timeseries, 10 times. We therefore need to loop through every timeseries, 10 times. 
At every run, we start by creating the dataset, and training a model. The remaining steps are similar to the examples which can be found in other notebooks such as the DVSGesture notebook.

In [ ]:
for repeat_id in range(repeat):
    for series_id in range(len(mg_parameters)):
        tau = mg_parameters.tau[series_id]

        # Load data using the parameters loaded from the csv file
        mg = MackeyGlass(tau = tau, 
                         lyaptime = mg_parameters.lyapunov_time[series_id],
                         constant_past = mg_parameters.initial_condition[series_id],
                         start_offset=start_offset_range[repeat_id].item(),
                         bin_window=1)

        # Split test and train set
        train_set = Subset(mg, mg.ind_train)
        test_set = Subset(mg, mg.ind_test)
        
        # Index of the hyperparamters for the current time-series
        ind_tau = esn_parameters.index[esn_parameters['tau'] == tau].tolist()[0]
    
        ## Fitting Model ##
        seed_id = repeat_id

        # Load the model with the parameters loaded from esn_parameters
        esn = EchoStateNetwork(in_channels=1, 
            reservoir_size = esn_parameters['reservoir_size'][ind_tau], 
            input_scale = torch.tensor([esn_parameters['scale_bias'][ind_tau], esn_parameters['scale_input'][ind_tau],],dtype = torch.float64), 
            connect_prob = esn_parameters['connect_prob'][ind_tau], 
            spectral_radius = esn_parameters['spectral_radius'][ind_tau],
            leakage = esn_parameters['leakage'][ind_tau], 
            ridge_param = esn_parameters['ridge_param'][ind_tau],
            seed_id = seed_id )

        esn.train()
        train_data, train_labels = train_set[:] # outputs (batch, bin_window, 1)
        warmup = 0.6 # in Lyapunov times
        warmup_pts = round(warmup*mg.pts_per_lyaptime)
        train_labels = train_labels[warmup_pts:]
        esn.fit(train_data, train_labels, warmup_pts)
        # save the model for later use
        torch.save(esn, 'neurobench/examples/mackey_glass/model_data/esn.pth')
         
        ## Load Model ##
        net = torch.load('neurobench/examples/mackey_glass/model_data/esn.pth')
        test_set_loader = DataLoader(test_set, batch_size=mg.testtime_pts, shuffle=False)

        # Wrap the model
        model = TorchModel(net)
    
        static_metrics = ["model_size", "connection_sparsity"]
        data_metrics = ["sMAPE", "activation_sparsity"]
    
        benchmark = Benchmark(model, test_set_loader, [], [], [static_metrics, data_metrics]) 
        results = benchmark.run()
        print(results)
        sMAPE_scores.append(results["sMAPE"])

print("Average sMAPE score accross all repeats and time series: ", sum(sMAPE_scores)/len(sMAPE_scores))

